In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score

# Load the CSV file
data = pd.read_csv('data_ANN.csv')

# Extract features (AT, V, AP, RH) into X
X = data[['AT', 'V', 'AP', 'RH']].values

# Extract target (PE) into y
y = data[['PE']].values

# Normalize input features
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)
X = (X - X_mean) / X_std  # Standardization

# Normalize target values
y_mean = y.mean()
y_std = y.std()
y = (y - y_mean) / y_std  # Standardization

# Define the neural network architecture
input_size = 4    # Number of input neurons (four features)
hidden_size = 12  # Increase the number of neurons in the hidden layer
output_size = 1   # Number of output neurons (one output)

# Initialize weights and biases
np.random.seed(42)
W1 = np.random.randn(input_size, hidden_size) * 0.01  # Small weight initialization
b1 = np.zeros((1, hidden_size))                       # Bias for hidden layer
W2 = np.random.randn(hidden_size, output_size) * 0.01 # Small weight initialization
b2 = np.zeros((1, output_size))                       # Bias for output layer

# Activation functions
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

# Forward propagation
def forward_propagation(X):
    Z1 = np.dot(X, W1) + b1
    A1 = relu(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = Z2  # Linear activation for output
    return Z1, A1, Z2, A2

# Loss function with L2 regularization
def compute_loss(y_true, y_pred, W1, W2, l2_lambda):
    m = y_true.shape[0]
    loss = np.mean((y_true - y_pred) ** 2)  # Mean Squared Error for regression
    l2_loss = (l2_lambda / (2 * m)) * (np.sum(np.square(W1)) + np.sum(np.square(W2)))
    return loss + l2_loss

# Backpropagation with L2 regularization
def backpropagation(X, y_true, Z1, A1, Z2, A2, W1, W2, l2_lambda):
    m = X.shape[0]
    dZ2 = (A2 - y_true)
    dW2 = (np.dot(A1.T, dZ2) / m) + (l2_lambda / m) * W2
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m
    dA1 = np.dot(dZ2, W2.T)
    dZ1 = dA1 * relu_derivative(Z1)
    dW1 = (np.dot(X.T, dZ1) / m) + (l2_lambda / m) * W1
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m
    return dW1, db1, dW2, db2

# Update parameters
def update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate):
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    return W1, b1, W2, b2

# Train the neural network
def train(X, y, epochs, learning_rate, l2_lambda):
    global W1, b1, W2, b2
    for epoch in range(epochs):
        Z1, A1, Z2, A2 = forward_propagation(X)
        loss = compute_loss(y, A2, W1, W2, l2_lambda)
        if epoch % 500 == 0:
            print(f"Epoch {epoch + 1}/{epochs} - Loss: {loss}")
        dW1, db1, dW2, db2 = backpropagation(X, y, Z1, A1, Z2, A2, W1, W2, l2_lambda)
        W1, b1, W2, b2 = update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)
    return W1, b1, W2, b2

# Make predictions
def predict(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_propagation(X)
    return A2

# Train the neural network with increased epochs and L2 regularization
W1, b1, W2, b2 = train(X, y, epochs=20000, learning_rate=0.01, l2_lambda=0.001)

# Make predictions
predictions = predict(X, W1, b1, W2, b2)

# Denormalize the predictions
denormalized_predictions = predictions * y_std + y_mean

# Compute R² score
r2 = r2_score(data[['PE']], denormalized_predictions)

print("Predictions:")
print(denormalized_predictions)
print(f"R² Score: {r2}")


Epoch 1/20000 - Loss: 0.9998922332268305
Epoch 501/20000 - Loss: 0.31334453195225037
Epoch 1001/20000 - Loss: 0.08523795061095167
Epoch 1501/20000 - Loss: 0.07641487772458068
Epoch 2001/20000 - Loss: 0.07233461932716803
Epoch 2501/20000 - Loss: 0.06987186789752706
Epoch 3001/20000 - Loss: 0.06820452563658203
Epoch 3501/20000 - Loss: 0.06699275170815719
Epoch 4001/20000 - Loss: 0.06608014468839657
Epoch 4501/20000 - Loss: 0.06539725755844503
Epoch 5001/20000 - Loss: 0.06487843768045062
Epoch 5501/20000 - Loss: 0.06447424136841061
Epoch 6001/20000 - Loss: 0.06415507142017234
Epoch 6501/20000 - Loss: 0.0638914434157395
Epoch 7001/20000 - Loss: 0.06366943561793381
Epoch 7501/20000 - Loss: 0.06348167816902309
Epoch 8001/20000 - Loss: 0.06329694899317712
Epoch 8501/20000 - Loss: 0.06312488575136838
Epoch 9001/20000 - Loss: 0.06297004367097499
Epoch 9501/20000 - Loss: 0.06282675796083499
Epoch 10001/20000 - Loss: 0.06269436558460545
Epoch 10501/20000 - Loss: 0.06258377873322031
Epoch 11001/20